<a href="https://colab.research.google.com/github/nikorose87/DJS-GA/blob/master/PCA_n_kPCA_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Young and elderly classification regardless gait speed in human motion analysis

## Prof. Nikolay Prieto 
*Universidad de San Buenaventura - Sede Bogotá*

We are going to apply Artificial Intelligence techniques to classify either the speed of the subject and the age group according to the data given from Horst's article [doi](https://data.mendeley.com/datasets/svx74xcrjr/3).

Also, literature about AI applied on the biomechanics field can be apprecieted in the Figueiredo's paper. In this opportunity we will build an AI algorithm able to classify if the person is young **y** or adult **a**, also in which velocity (Too slow **XS**, Slow **S**, Natural Speed **Natural**, More than normal **M** and fast **L**) 

Let us explain the procedure:



In [19]:
!wget https://www.dropbox.com/s/bqodz4qn7zhi3cz/Horst_mean.csv
!wget https://www.dropbox.com/s/2bfc8vne4rx7o9d/Gait_subject_info.csv
!wget https://www.dropbox.com/s/8g3kimgzy7j4o5x/Horst_Nature_paper.csv

--2020-10-20 22:20:30--  https://www.dropbox.com/s/bqodz4qn7zhi3cz/Horst_mean.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bqodz4qn7zhi3cz/Horst_mean.csv [following]
--2020-10-20 22:20:30--  https://www.dropbox.com/s/raw/bqodz4qn7zhi3cz/Horst_mean.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8da8657104554d034310271ef1.dl.dropboxusercontent.com/cd/0/inline/BBpCPgFrZQezbVNCSsrLp-qzCmBat9rEjTQpyX-RNwqJ3vgeKf7400B5EKW0PihGySLN5G0eyEDrYrLzPQhyU2ADSsN6stpIzS0k1l9fV9bO3fq4ifGs6cInXiEUeaPdIDI/file# [following]
--2020-10-20 22:20:30--  https://uc8da8657104554d034310271ef1.dl.dropboxusercontent.com/cd/0/inline/BBpCPgFrZQezbVNCSsrLp-qzCmBat9rEjTQpyX-RNwqJ3vgeKf7400B5EKW0PihGySLN5G0eyEDrYrLzPQhyU2ADSsN6stpIzS0k1l9fV9bO

In [20]:
#Checking if the file is already there
import sys
import os
#print(sys.path)
!pwd
!ls

/content
 Gait_subject_info.csv	   Horst_mean.csv.1	  Horst_Nature_paper.csv
 Gait_subject_info.csv.1   Horst_mean.csv.2	  Horst_Nature_paper.csv.1
 Gait_subject_info.csv.2   Horst_mean.csv.3	 'Horst_Nature_paper.csv?dl=0'
 Horst_mean.csv		  'Horst_mean.csv?dl=1'   sample_data


# Some concepts about Human Gait

## How the planes in human motion are named

![human planes](https://drive.google.com/uc?id=1icMDhWCw_1s4FKYPgSEJYIUX7_Wwo3x1)

## Terminology in movements
![movements in gait](https://drive.google.com/uc?id=1hTtXIwNR7VXbimmYdlbwW4t1eQPFSnXn)

## Terminology only for ankle foot
![foot movements](https://drive.google.com/uc?id=1_kTatzluVRXAC2D1XhaP7X48M02MkUkZ)

## How the data was processed 

![Opensim](https://raw.githubusercontent.com/opensim-org/opensim-core/master/doc/images/opensim_running.gif)

With Opensim we have obtained all the kinematic and kinetic data for each trial with the following procedure:

![Kinematics](https://simtk-confluence.stanford.edu:8443/download/attachments/29164184/worddav18beee95e9328969b6f5ee05a6c736f0.png?version=1&modificationDate=1530846003979&api=v2)





In [21]:
import warnings
import numpy as np
import pandas as pd
from bokeh.layouts import row, column
from bokeh.palettes import Spectral6, Set3
from bokeh.plotting import figure, show, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.glyphs import Text
from bokeh.io import reset_output, output_notebook
from scipy.integrate import simps
import scipy.optimize as optimization
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from pathlib import PurePath
import os
warnings.filterwarnings('ignore')
output_notebook()
##Cleaning and selecting the data
reset_output()

Horst_mean = pd.read_csv('Horst_mean.csv', header=[0,1], index_col=[0,1])

output_label = pd.read_csv('Gait_subject_info.csv')
## Building the subtables

Let us check how the data is behaving:

In [22]:
Horst_mean.sample(10)

Subject                                        S01  ...         S57
                                              -1sd  ...        +1sd
Feature                 Gait cycle [%]              ...            
wrist_flex_r_moment     0.8485            0.013243  ...    0.065980
hip_rotation_r_moment   0.5657           -2.367678  ...   -1.508262
pelvis_ty_force         0.4242         -183.552028  ...  132.037639
wrist_dev_r_moment      0.6263           -0.174763  ...   -0.090060
ground_force_2_v_3      0.4747           -0.097121  ...   -0.067405
elbow_flex_l            0.5253           43.091793  ...   24.274547
arm_add_l               0.3232          -25.099829  ...   -7.496365
pelvis_tx_force         0.1818           42.905694  ...   39.749161
hip_rotation_r          0.8687           -8.453116  ...    5.655475
knee_angle_l_beta_force 0.0202           -0.048685  ...   -0.049565

[10 rows x 171 columns]

In [24]:
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import math
from itertools import combinations

#Setting plot parameters
plt.rcParams.update({'font.size': 22})
# Cleaning the plots
plt.cla()
plt.clf()
plt.close()
sd=False
def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)


def get_mult_num(integer):
  """
  From https://stackoverflow.com/questions/54556363/finding-two-integers-that-multiply-to-20-can-i-make-this-code-more-pythonic
  """
  given_comb = [1,1,2,2,3,3,4,4,5,5,6,6,7,7]
  return [(i, j) for i, j in list(combinations(given_comb,2)) if i * j == integer]
  
def subpl(integer, a=9, dif=3):
  """
  Function to generate the best combination among a number of plots
  It does not work, use_get_mult_num instead
  """
  a_init = a
  b_init = b = a - dif 
  try:
    while lcm(a,b) != integer and a+b >=0:
      print(a,b)
      a -= 1
      b -= 1
  except (ValueError, ZeroDivisionError) as e:
    print('Determining the suitable division could not be done, trying bypairs')
    dif += 1
    a,b = subpl(integer, a=a_init, dif=dif)
  return a,b


In [25]:
def gait_plot(df_, label, sd=False, title=False):
  # Cleaning the plots
  plt.cla()
  plt.clf()
  plt.close()
  
  # Getting the first index in rows
  indexes= df_.index.get_level_values(0).unique()
  #Gait cycle
  x = df_.index.get_level_values(1).unique()
  #nrows, ncols = subpl(len(lab))
  nrows, ncols = get_mult_num(len(indexes))[-1]
  fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(nrows*ncols*2, ncols*nrows), sharex=True)
  # "ravel" flattens the numpy array without making a copy
  for j, ax in enumerate(axs.ravel()):
    for i in label:
      mean = df_[i,'mean'][indexes[j]].values
      sd_min = df_[i,'SD-'][indexes[j]].values
      sd_max = df_[i,'SD+'][indexes[j]].values
      ax.plot(x, mean, '-')
      if sd:
        ax.fill_between(x, sd_min, sd_max, alpha=0.2)
      ax.set_xlabel('Gait cycle [%]')
      ax.set_ylabel(indexes[j]+' [deg]')
  ax.legend(label, loc='center left', bbox_to_anchor=(1, 0.5))
  if title:  plt.suptitle(title)
  plt.show()
  # ax.plot(x, y, 'o', color='tab:brown')
  return

In [27]:
labels = Horst_mean.
gait_plot(Horst_mean, labels[:3], sd=True, title='Angles for youth population')

NameError: ignored

In [ ]:
df_ = complete_data[sheet_names[3]]
gait_plot(df_, [labels[0], labels[9]], sd=False, title='GRF for youth people')

In [ ]:
##Calling by multi row
idx = pd.IndexSlice
indexes= df_.index.get_level_values(0).unique()
df_['Natural_y'].sample()

In [ ]:
#Calling by multi index and multirow
df_.loc[idx[:, :], idx[:, 'mean']]


## How to use the PCA to classify according to the data

We are going to base on the work done by **Eskofier et al** [3] which abstract is:


> *The classification of gait patterns has great potential as a diagnostic tool, for example, for the diagnosis of injury or to identify at-risk gait in the elderly. The purpose of the paper is to present a method for classifying group differences in gait pattern by using the complete spatial and temporal information of the segment motion quantified by the markers. The classification rates that are obtained are compared with previous studies using conventional classification features. For our analysis, 37 three-dimensional marker trajectories were collected from each of our 24 young and 24 elderly female subjects while they were walking on a treadmill. Principal component analysis was carried out on these trajectories to retain the spatial and temporal information in the markers. Using a Support Vector Machine with a linear kernel, a classification rate of 95.8% was obtained. This classification approach also allowed visualisation of the contribution of individual markers to group differentiation in position and time. The approach made no specific assumptions and did not require prior knowledge of specific time points in the gait cycle. It is therefore directly applicable for group classification tasks in any study involving marker measurements.*



### How the population was set up in the experiment

![population](https://drive.google.com/uc?id=1tPM95BR0RKJXewVQr_FArLogsJabCawz)

### Which were the variables implied in the human gait analysis

![markers](https://drive.google.com/uc?id=1V6qAzovgWS_Hlu8_66dfqGSU_-nuWh9C)

![marker vi](https://drive.google.com/uc?id=1fySK5muO4eubvpkxFPKQEseR-6GNwvxW)

### Which classification algorithm was applied to the problem

![marker viz](https://drive.google.com/uc?id=1Tc0JxEmy-_JFW_0K2SqbwLiEKyCNN52E)

# Your Task


1.   Be familiar with the Anthropometric concepts implied in the dataset.
2.   Rearrange the data so that you can implement a feature extraction process.
Implement a feasible and believable feature extraction process using PCA or kPCA.
3. Perform a multiclass classification process  using your prefered method. 
Determine the number of Principal Components in which you get the highest accuracy.
4. Get your own conclusions



**Plus:** Use the feature importance process to know the most important variables in the process.





[1] G. Bovi, M. Rabuffetti, P. Mazzoleni, M. Ferrarin, A multiple-task gait analysis approach: Kinematic, kinetic and EMG reference data for healthy young and adult subjects, Gait Posture. 33 (2011) 6–13. https://doi.org/10.1016/j.gaitpost.2010.08.009.

[2] J. Figueiredo, C.P. Santos, J.C. Moreno, Automatic recognition of gait patterns in human motor disorders using machine learning: A review, Med. Eng. Phys. 53 (2018) 1–12. https://doi.org/10.1016/j.medengphy.2017.12.006.

[3] B.M. Eskofier, P. Federolf, P.F. Kugler, B.M. Nigg, Marker-based classification of young-elderly gait pattern differences via direct PCA feature extraction and SVMs, Comput. Methods Biomech. Biomed. Engin. 16 (2013) 435–442. https://doi.org/10.1080/10255842.2011.624515.

[4] M.W. Whittle, Gait Analysis an Introduction, 4th ed., Elsevier, 2007.
